In [4]:
print(tf.__version__)

2.17.0


In [2]:
import re
import constants
import os
import pandas as pd
import multiprocessing
import time
from tqdm import tqdm
import numpy as np
from pathlib import Path
from functools import partial
import urllib.request
from PIL import Image
import logging

# Set up logging for better debugging and monitoring
logging.basicConfig(level=logging.WARNING, format='%(asctime)s - %(levelname)s - %(message)s')  # Set logging level to WARNING

def common_mistake(unit):
    """
    Corrects common mistakes in units (e.g., different spellings or typos).
    """
    if unit in constants.allowed_units:
        return unit
    if unit.replace('ter', 'tre') in constants.allowed_units:
        return unit.replace('ter', 'tre')
    if unit.replace('feet', 'foot') in constants.allowed_units:
        return unit.replace('feet', 'foot')
    return unit

def parse_string(s):
    """
    Parses and validates a prediction string, ensuring it follows the format 'number unit'.
    """
    s_stripped = "" if s is None or str(s) == 'nan' else s.strip()
    if s_stripped == "":
        return None, None
    pattern = re.compile(r'^-?\d+(\.\d+)?\s+[a-zA-Z\s]+$')
    if not pattern.match(s_stripped):
        raise ValueError("Invalid format in {}".format(s))
    parts = s_stripped.split(maxsplit=1)
    number = float(parts[0])
    unit = common_mistake(parts[1])
    if unit not in constants.allowed_units:
        raise ValueError("Invalid unit [{}] found in {}. Allowed units: {}".format(
            unit, s, constants.allowed_units))
    return number, unit

def create_placeholder_image(image_save_path):
    """
    Creates a black placeholder image if downloading fails.
    """
    try:
        placeholder_image = Image.new('RGB', (100, 100), color='black')
        placeholder_image.save(image_save_path)
        logging.warning(f"Placeholder image created at {image_save_path}")
    except Exception as e:
        logging.error(f"Error creating placeholder image: {e}")

def download_image(image_link, save_folder, retries=5, delay=5):
    """
    Downloads an image from a given link. If the download fails after the specified retries,
    a placeholder image is created.
    """
    if not isinstance(image_link, str) or not image_link.strip():  # Check for empty or None links
        logging.warning(f"Invalid or empty image link: {image_link}")
        return

    filename = Path(image_link).name
    image_save_path = os.path.join(save_folder, filename)

    if os.path.exists(image_save_path):
        return  # Skip logging for existing images

    for attempt in range(retries):
        try:
            urllib.request.urlretrieve(image_link, image_save_path)  # Correct use of urllib.request
            if validate_image(image_save_path):
                logging.info(f"Image successfully downloaded: {image_save_path}")
                return
            else:
                logging.warning(f"Invalid image detected, retrying... ({attempt + 1}/{retries})")
        except Exception as e:
            logging.error(f"Error downloading image {image_link}: {e}")
            time.sleep(delay)

    # logging.error(f"Failed to download image after {retries} retries: {image_link}")
    create_placeholder_image(image_save_path)  # Create a black placeholder image for invalid links/images

def validate_image(image_path):
    """
    Checks if the downloaded image is valid and not corrupted.

    Args:
    - image_path (str): Path to the image file.

    Returns:
    - bool: True if the image is valid, False otherwise.
    """
    try:
        with Image.open(image_path) as img:
            img.verify()  # Verify that it is an image
        # Reload the image to check if it can be converted to an array (optional, can be skipped)
        img = Image.open(image_path)
        img.load()
        return True
    except Exception as e:
        logging.error(f"Image validation failed for {image_path}: {e}")
        return False

def preprocess_image(image_path, target_size=(224, 224)):
    """
    Preprocesses the image for model prediction.
    Includes resizing, normalization, etc.

    Args:
    - image_path (str): Path to the image.
    - target_size (tuple): Target size for resizing (width, height).

    Returns:
    - np.array: Preprocessed image ready for prediction.
    """
    try:
        img = Image.open(image_path).convert('RGB')  # Convert to RGB
        img = img.resize(target_size)
        img = np.array(img) / 255.0  # Normalize pixel values to [0, 1]
        return img
    except Exception as e:
        logging.error(f"Error preprocessing image {image_path}: {e}")
        return None

def download_images(image_links, download_folder, allow_multiprocessing=True, max_workers=20):
    """
    Downloads multiple images using multiprocessing for efficiency.
    Adjusts the number of workers to avoid Windows handle limitations.
    """
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    if allow_multiprocessing:
        download_image_partial = partial(
            download_image, save_folder=download_folder, retries=3, delay=3)

        with multiprocessing.Pool(min(max_workers, 20)) as pool:  # Limit number of workers to avoid Windows handle limits
            list(tqdm(pool.imap(download_image_partial, image_links), total=len(image_links)))
            pool.close()
            pool.join()
    else:
        for image_link in tqdm(image_links, total=len(image_links)):
            download_image(image_link, save_folder=download_folder, retries=2, delay=2)

def load_and_download_images(csv_file_path, download_folder, sample_size=None):
    """
    Loads image links from a CSV file, samples a subset, and downloads them to the specified folder.

    Args:
    - csv_file_path (str): Path to the CSV file containing image links.
    - download_folder (str): Folder to download images to.
    - sample_size (int, optional): Number of samples to use for testing. Default is None (use full dataset).
    """
    df = pd.read_csv(csv_file_path, on_bad_lines='skip')
    if sample_size is not None:
        df = df.sample(n=sample_size, random_state=42)  # Sample a subset for quick testing
    image_links = df['image_link'].tolist()

    # Ensure the folder exists
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    # Download images
    download_images(image_links, download_folder)

if __name__ == "__main__":
    # Example usage: download images from train and test datasets with sampling
    train_csv_path = 'dataset/cleaned_train2.csv'
    test_csv_path = 'dataset/test.csv'
    sample_test_csv_path = 'dataset/sample_test.csv'
    train_images_folder = 'images/train'
    sample_test_images_folder = 'images/sample_test'
    test_images_folder = 'images/test'

    # Download a small subset of images for testing
    sample_size_train = None
    sample_size_test = None

    # Load image links from CSV files and download them
    load_and_download_images(train_csv_path, train_images_folder, sample_size=sample_size_train)
    # load_and_download_images(test_csv_path, test_images_folder, sample_size=sample_size_test)
    # load_and_download_images(sample_test_csv_path, sample_test_images_folder, sample_size=None)


  0%|          | 515/263511 [00:20<2:32:50, 28.68it/s]ERROR:root:Error downloading image https://m.media-amazon.com/images/I/1yw53vfQtS.jpg: HTTP Error 400: Bad Request
ERROR:root:Image validation failed for images/train/71yMHX+ppyL.jpg: image file is truncated (6 bytes not processed)
ERROR:root:Error downloading image https://m.media-amazon.com/images/I/1yw53vfQtS.jpg: HTTP Error 400: Bad Request
  3%|▎         | 7722/263511 [04:34<3:05:23, 23.00it/s]ERROR:root:Error downloading image https://m.media-amazon.com/images/I/DzP2RMRQO0.jpg: HTTP Error 400: Bad Request
ERROR:root:Error downloading image https://m.media-amazon.com/images/I/DzP2RMRQO0.jpg: HTTP Error 400: Bad Request
  7%|▋         | 17255/263511 [09:47<2:20:09, 29.28it/s]ERROR:root:Error downloading image https://m.media-amazon.com/images/I/lwd2cSmT2ux.jpg: HTTP Error 404: Not Found
ERROR:root:Error downloading image https://m.media-amazon.com/images/I/lwd2cSmT2ux.jpg: HTTP Error 404: Not Found
  8%|▊         | 21718/263511 

KeyboardInterrupt: 

In [3]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score
from tensorflow.keras.layers import Input, Dense, LSTM, Conv2D, MaxPooling2D, Reshape, Bidirectional, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from PIL import Image
from tqdm import tqdm
from urllib.parse import urlparse
import string
import re

# Set up paths
TRAIN_CSV_PATH = 'dataset/cleaned_train2.csv'
TEST_CSV_PATH = 'dataset/test.csv'
TRAIN_IMAGES_DIR = 'images/train'
TEST_IMAGES_DIR = 'images/test'
MODEL_SAVE_PATH = '/content/drive/MyDrive/Model/ocr_model.h5'
OUTPUT_CSV_PATH = '/content/drive/MyDrive/Model/test_out.csv'

# Set constants
IMAGE_SIZE = 250  # Maximum size for any dimension
MAX_TEXT_LENGTH = 35  # Adjust based on your data
BATCH_SIZE = 128
EPOCHS = 20
SAMPLE_SIZE = 50000  # Number of samples to use for initial training; set to None to use the full dataset

# Load entity-unit mappings with abbreviations and plural forms
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'maximum_weight_recommendation': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre', 'cubic foot', 'cubic inch', 'cup', 'decilitre', 'fluid ounce', 'gallon',
                    'imperial gallon', 'litre', 'microlitre', 'millilitre', 'pint', 'quart'}
}

# Allowed characters for OCR predictions (including digits and units)
ALLOWED_CHARACTERS = string.digits + " " + "".join(set().union(*entity_unit_map.values()))

# Function to extract the image filename from the image link
def get_image_filename(image_link):
    parsed_url = urlparse(image_link)
    return os.path.basename(parsed_url.path)

# Function to preprocess images with aspect ratio preservation and padding
def preprocess_image(image_path):
    img = Image.open(image_path).convert('L')  # Convert to grayscale

    # Resize while keeping aspect ratio
    img.thumbnail((IMAGE_SIZE, IMAGE_SIZE), Image.LANCZOS)

    # Create a new square image with a black background
    new_img = Image.new('L', (IMAGE_SIZE, IMAGE_SIZE), color=0)

    # Paste the resized image onto the center of the new square image
    new_img.paste(img, ((IMAGE_SIZE - img.width) // 2, (IMAGE_SIZE - img.height) // 2))

    img = np.array(new_img) / 255.0  # Normalize to [0, 1]
    img = np.expand_dims(img, axis=-1)  # Add channel dimension
    return img

# Load training data
def load_data(csv_path, images_dir, sample_size=None):
    df = pd.read_csv(csv_path, on_bad_lines='skip')  # Skips bad lines

    # Use only a subset of the data if sample_size is specified
    if sample_size is not None:
        df = df.sample(n=sample_size, random_state=42)  # Randomly sample data for initial training

    images = []
    labels = []

    for idx, row in tqdm(df.iterrows(), total=len(df)):
        image_path = os.path.join(images_dir, get_image_filename(row['image_link']))
        if os.path.exists(image_path):
            try:
                img = preprocess_image(image_path)
                images.append(img)
                labels.append(encode_text(row['entity_value']))  # Encode labels
            except (OSError, IOError) as e:
                # Skip images that are truncated or have issues
                print(f"Skipping file {image_path}: {e}")
                continue

    return np.array(images), np.array(labels)

# Encode text into numerical representation for training
def encode_text(text):
    try:
        number, unit = text.split()
        unit_index = list(entity_unit_map.keys()).index(unit)
        # Return a one-hot encoded vector for the unit and number
        one_hot = np.zeros(len(entity_unit_map.keys()))
        one_hot[unit_index] = 1
        return [float(number)] + one_hot.tolist()
    except (ValueError, IndexError) as e:
        # If parsing fails, return zeros
        return [0.0] + [0] * len(entity_unit_map.keys())

# Decode numerical representation back to text
def decode_text(prediction):
    # If prediction is a 1D array, reshape it to 2D for consistent processing
    if len(prediction.shape) == 1:
        prediction = np.expand_dims(prediction, axis=0)

    if len(prediction[0]) == 1 + len(entity_unit_map.keys()):  # Ensure the prediction length is valid
        number = prediction[0][0]  # Get the predicted number
        unit_idx = np.argmax(prediction[0][1:])  # Get the index of the predicted unit
        unit = list(entity_unit_map.keys())[unit_idx]  # Map the index back to the unit
        return f"{number:.2f} {unit}"
    else:
        return ""  # Return empty if the prediction is invalid

# Define the model architecture using CRNN (Convolutional Recurrent Neural Network)
def build_crnn_model(input_shape):
    input_img = Input(shape=input_shape, name='image_input')

    # Convolutional layers
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    # Reshape the output to 3D for LSTM layers
    time_steps, features = x.shape[1] * x.shape[2], x.shape[3]
    x = Reshape(target_shape=(time_steps, features))(x)

    # Recurrent layers
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = Bidirectional(LSTM(128, return_sequences=False))(x)

    # Fully connected layer for regression (first output is the number, the rest are the one-hot encoded unit vectors)
    output = Dense(1 + len(entity_unit_map.keys()), activation='linear')(x)

    model = Model(inputs=input_img, outputs=output)
    return model

# Custom loss function for regression
def custom_loss(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred))

# Compile and train the model
def train_model(model, train_images, train_labels, val_images, val_labels):
    model.compile(optimizer=Adam(learning_rate=1e-4), loss=custom_loss, metrics=['mae'])

    # Save the best model based on validation loss
    checkpoint = ModelCheckpoint(MODEL_SAVE_PATH, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

    # Reduce learning rate when a metric has stopped improving
    lr_reduction = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, min_lr=1e-6, verbose=1)

    # Using data augmentation to improve generalization
    train_datagen = ImageDataGenerator(rotation_range=2, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.01, zoom_range=[0.9, 1.25])
    val_datagen = ImageDataGenerator(rescale=1./255)

    # Fit the model with data augmentation for training set and rescaling for validation set
    model.fit(
        train_datagen.flow(train_images, train_labels, batch_size=BATCH_SIZE),
        validation_data=val_datagen.flow(val_images, val_labels, batch_size=BATCH_SIZE),
        epochs=EPOCHS,
        callbacks=[checkpoint, lr_reduction]
    )

    return model

# Main function
if __name__ == "__main__":
    # Load and preprocess data
    images, labels = load_data(TRAIN_CSV_PATH, TRAIN_IMAGES_DIR, sample_size=SAMPLE_SIZE)

    # Split into training and validation sets
    train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size=0.2, random_state=42)
    input_shape = (IMAGE_SIZE, IMAGE_SIZE, 1)

    # Build, train, and evaluate model
    model = build_crnn_model(input_shape)
    model = train_model(model, train_images, train_labels, val_images, val_labels)


 20%|█▉        | 9769/50000 [01:09<03:50, 174.77it/s]

Skipping file images/train/71Z8jYq8OLL.jpg: cannot identify image file '/content/images/train/71Z8jYq8OLL.jpg'


 45%|████▍     | 22389/50000 [02:39<03:48, 120.58it/s]

Skipping file images/train/81S2Z43PCvL.jpg: cannot identify image file '/content/images/train/81S2Z43PCvL.jpg'


100%|██████████| 50000/50000 [05:50<00:00, 142.75it/s]


Epoch 1/20
84/84 [==============================] - ETA: 0s - loss: 1.1037e-04 - mae: 0.0050
Epoch 1: val_loss improved from inf to 0.00000, saving model to /content/drive/MyDrive/Model/ocr_model.h5
84/84 [==============================] - 565s 7s/step - loss: 1.1037e-04 - mae: 0.0050 - val_loss: 2.8611e-06 - val_mae: 0.0014 - lr: 1.0000e-04
Epoch 2/20


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


84/84 [==============================] - ETA: 0s - loss: 2.5269e-06 - mae: 9.6979e-04
Epoch 2: val_loss improved from 0.00000 to 0.00000, saving model to /content/drive/MyDrive/Model/ocr_model.h5
84/84 [==============================] - 561s 7s/step - loss: 2.5269e-06 - mae: 9.6979e-04 - val_loss: 2.7077e-07 - val_mae: 4.2875e-04 - lr: 1.0000e-04
Epoch 3/20
84/84 [==============================] - ETA: 0s - loss: 1.5777e-06 - mae: 7.4001e-04
Epoch 3: val_loss improved from 0.00000 to 0.00000, saving model to /content/drive/MyDrive/Model/ocr_model.h5
84/84 [==============================] - 561s 7s/step - loss: 1.5777e-06 - mae: 7.4001e-04 - val_loss: 5.9052e-08 - val_mae: 1.8119e-04 - lr: 1.0000e-04
Epoch 4/20
84/84 [==============================] - ETA: 0s - loss: 1.2405e-06 - mae: 6.6032e-04
Epoch 4: val_loss improved from 0.00000 to 0.00000, saving model to /content/drive/MyDrive/Model/ocr_model.h5

Epoch 4: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.
84/84 [

In [1]:
# Evaluate Model and Calculate F1 Score
def evaluate_model(model, val_images, val_labels):
    predictions = model.predict(val_images)
    predicted_texts = [decode_text(pred) for pred in predictions]

    # Convert true labels to their original format
    true_texts = [decode_text(true) for true in val_labels]

    # Calculate F1 Score
    f1 = f1_score(true_texts, predicted_texts, average='weighted', zero_division=1)
    precision = precision_score(true_texts, predicted_texts, average='weighted', zero_division=1)
    recall = recall_score(true_texts, predicted_texts, average='weighted', zero_division=1)

    print(f"F1 Score: {f1:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")

# Evaluate the model and calculate F1 score
evaluate_model(model, val_images, val_labels)


NameError: name 'model' is not defined

In [3]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import f1_score, precision_score, recall_score
from tensorflow.keras.models import load_model
from PIL import Image
from urllib.parse import urlparse
import string

# Load entity-unit mappings with abbreviations and plural forms
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'maximum_weight_recommendation': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre', 'cubic foot', 'cubic inch', 'cup', 'decilitre', 'fluid ounce', 'gallon',
                    'imperial gallon', 'litre', 'microlitre', 'millilitre', 'pint', 'quart'}
}

# Function to extract the image filename from the image link
def get_image_filename(image_link):
    parsed_url = urlparse(image_link)
    return os.path.basename(parsed_url.path)

# Function to preprocess images with aspect ratio preservation and padding
def preprocess_image(image_path, image_size=250):
    img = Image.open(image_path).convert('L')  # Convert to grayscale

    # Resize while keeping aspect ratio
    img.thumbnail((image_size, image_size), Image.LANCZOS)

    # Create a new square image with a black background
    new_img = Image.new('L', (image_size, image_size), color=0)

    # Paste the resized image onto the center of the new square image
    new_img.paste(img, ((image_size - img.width) // 2, (image_size - img.height) // 2))

    img = np.array(new_img) / 255.0  # Normalize to [0, 1]
    img = np.expand_dims(img, axis=-1)  # Add channel dimension
    return img

# Encode text into numerical representation for training
def encode_text(text):
    try:
        number, unit = text.split()
        unit_index = list(entity_unit_map.keys()).index(unit)
        # Return a one-hot encoded vector for the unit and number
        one_hot = np.zeros(len(entity_unit_map.keys()))
        one_hot[unit_index] = 1
        return [float(number)] + one_hot.tolist()
    except (ValueError, IndexError) as e:
        # If parsing fails, return zeros
        return [0.0] + [0] * len(entity_unit_map.keys())

# Decode numerical representation back to text
def decode_text(prediction):
    # If prediction is a 1D array, reshape it to 2D for consistent processing
    if len(prediction.shape) == 1:
        prediction = np.expand_dims(prediction, axis=0)

    if len(prediction[0]) == 1 + len(entity_unit_map.keys()):  # Ensure the prediction length is valid
        number = prediction[0][0]  # Get the predicted number
        unit_idx = np.argmax(prediction[0][1:])  # Get the index of the predicted unit
        unit = list(entity_unit_map.keys())[unit_idx]  # Map the index back to the unit
        return f"{number:.2f} {unit}"
    else:
        return ""  # Return empty if the prediction is invalid

# Load test data
def load_test_data(csv_path, images_dir, start_index=0, num_files=50):
    df = pd.read_csv(csv_path, on_bad_lines='skip')
    df = df.iloc[start_index:start_index + num_files]  # Select the specified range

    images = []
    labels = []

    for idx, row in df.iterrows():
        image_path = os.path.join(images_dir, get_image_filename(row['image_link']))
        if os.path.exists(image_path):
            try:
                img = preprocess_image(image_path)
                images.append(img)
                labels.append(encode_text(row['entity_value']))  # Encode labels
            except (OSError, IOError) as e:
                # Skip images that are truncated or have issues
                print(f"Skipping file {image_path}: {e}")
                continue

    return np.array(images), np.array(labels)

# Evaluate Model and Calculate F1 Score
def evaluate_model(model_path, csv_path, images_dir, start_index=0, num_files=50):
    # Load model
    model = load_model(model_path, compile=False)

    # Load test data
    val_images, val_labels = load_test_data(csv_path, images_dir, start_index, num_files)

    # Predict on test data
    predictions = model.predict(val_images)
    predicted_texts = [decode_text(pred) for pred in predictions]

    # Convert true labels to their original format
    true_texts = [decode_text(true) for true in val_labels]

    # Calculate F1 Score
    f1 = f1_score(true_texts, predicted_texts, average='weighted', zero_division=1)
    precision = precision_score(true_texts, predicted_texts, average='weighted', zero_division=1)
    recall = recall_score(true_texts, predicted_texts, average='weighted', zero_division=1)

    print(f"F1 Score: {f1:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")

# Example usage
if __name__ == "__main__":
    model_path = '/content/drive/MyDrive/Model/ocr_model50k-20.h5'  # Path to your saved model
    csv_path = 'dataset/cleaned_train2.csv'  # Path to the CSV file
    images_dir = 'images/train'  # Directory containing images
    start_index = 110000  # Starting index of the CSV data to test
    num_files = 1000  # Number of files to test

    evaluate_model(model_path, csv_path, images_dir, start_index, num_files)


ValueError: Unrecognized keyword arguments passed to LSTM: {'time_major': False}

In [ ]:
# Function to predict and generate the output for the test dataset
def predict_and_generate_output(model, test_csv_path, images_dir, output_csv_path):
    test_df = pd.read_csv(test_csv_path)
    results = []

    for _, row in tqdm(test_df.iterrows(), total=len(test_df)):
        image_path = os.path.join(images_dir, get_image_filename(row['image_link']))
        if os.path.exists(image_path):
            img = preprocess_image(image_path)
            prediction = model.predict(np.expand_dims(img, axis=0))
            decoded_text = decode_text(prediction[0])  # Decode prediction to text
            results.append((row['index'], decoded_text))
        else:
            results.append((row['index'], ""))

    # Save predictions to CSV
    pd.DataFrame(results, columns=['index', 'prediction']).to_csv(output_csv_path, index=False)

# Predict on test images and generate the output CSV file
predict_and_generate_output(model, TEST_CSV_PATH, TEST_IMAGES_DIR, OUTPUT_CSV_PATH)
